In [4]:
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/"

Mounted at /content/drive


In [20]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [6]:
train_ds = pd.read_csv("/content/drive/MyDrive/Dataset/labels_train.csv")
train_ds.head()

,Unnamed: 0,Image path,Class,Weight
0,0,Dataset/Train/FBO1/cacao-31.jpg,FBO1,1.66
1,1,Dataset/Train/FBO1/cacao-32.jpg,FBO1,1.27
2,2,Dataset/Train/FBO1/cacao-33.jpg,FBO1,1.79
3,3,Dataset/Train/FBO1/cacao-34.jpg,FBO1,1.25
4,4,Dataset/Train/FBO1/cacao-35.jpg,FBO1,1.85


In [ ]:
from scipy.ndimage import binary_fill_holes
from skimage.measure import regionprops
from skimage.measure import label

def add_padding(image, target_size):
    # Get the current size of the image
    h, w = image.shape[:2]

    # Calculate padding sizes
    top = bottom = (target_size[0] - h) // 2
    left = right = (target_size[1] - w) // 2

    # Adjust if the padding is not even (for odd target size)
    if h % 2 != 0:
        bottom += 1
    if w % 2 != 0:
        right += 1

    # Add padding (black by default)
    padded_image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))

    return padded_image

def process_image(image):

    # Resize once and convert to grayscale
    image_resized = cv2.resize(image, (500, 300))

    image_gray = cv2.cvtColor(image_resized, cv2.COLOR_RGB2GRAY)

    # Thresholding and hole filling
    _, img = cv2.threshold(image_gray, 0, 1, cv2.THRESH_OTSU)
    img = 1 - img
    img = binary_fill_holes(img)

    # Calculate largest bounding box using regionprops
    lab, num = label(img, return_num=True)
    max_area = 0
    bbox = []

    for i in range(1, num + 1):
        object_region = (lab == i).astype('uint8')
        prop = regionprops(object_region)[0]
        area = prop.area
        if area > max_area:
            max_area = area
            bbox = prop.bbox

    # Crop the image and apply the mask
    img_cropped = image_resized[bbox[0]: bbox[2], bbox[1]: bbox[3]]
    mask_cropped = img[bbox[0]: bbox[2], bbox[1]:bbox[3]]
    img_cropped = img_cropped * mask_cropped[..., None]

    padded_image = add_padding(img_cropped, [250, 150])
    padded_image = padded_image / 255.0

    return padded_image

In [41]:
# sizes = []
from tqdm import tqdm

for i in tqdm(range(len(train_ds))):

  image = cv2.imread(BASE_PATH + train_ds.iloc[i]["Image path"])
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  image = process_image(image)
  path = "Dataset/processed/" + str(train_ds.iloc[i]["Image path"]).replace("Dataset/", "")
  cv2.imwrite(BASE_PATH + path, image)

  2%|▏         | 25/1612 [00:34<36:58,  1.40s/it]


KeyboardInterrupt: 

In [21]:
import numpy as np
arr = np.array(sizes)
df = pd.DataFrame(arr, columns=["Height", "Width", "Channels"])
df.max()

,0
Height,194
Width,126
Channels,3
